In [9]:
import torch
import torch.nn.functional as F
import torch.nn as nn
import torchvision
import torchvision.datasets as datasets
import matplotlib.pyplot as plt
import numpy as np
import pickle
from torch import Tensor
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import matplotlib.pyplot as plt
import pandas as pd
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from scipy.fft import rfft, rfftfreq, fft, fftfreq
import scipy
import time
import copy
from google.colab import drive

drive.mount('/content/drive')

# Load in Data
with open('/content/drive/MyDrive/CSE 481 Capstone/processed_data.npy', 'rb') as f:
    data = np.load(f)
print(data.shape)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
(17920, 4, 32, 32)


In [10]:
# Load in Labels
labels = pd.read_excel("/content/drive/MyDrive/CSE 481 Capstone/metadata_xls/participant_ratings.xls")
print(labels.head())
sub_labels = []
for i in range(len(labels)):
  sub_labels.append([labels.loc[i, 'Valence'], labels.loc[i, 'Arousal']])
sub_labels = np.array(sub_labels)
print(sub_labels.shape)
print(sub_labels)
sub_labels_2 = np.zeros((len(sub_labels)))
for i in range(len(sub_labels)):
  instance = sub_labels[i]
  valence = instance[0]
  arousal = instance[1]
  if (valence < 5 and arousal < 5):
    sub_labels_2[i] = 0
  elif (valence < 5 and arousal >= 5):
    sub_labels_2[i] = 1
  elif (valence >= 5 and arousal < 5):
    sub_labels_2[i] = 2
  else:
    sub_labels_2[i] = 3
#lb = preprocessing.LabelBinarizer()
#sub_labels_2 = lb.fit_transform(sub_labels_2)
print(sub_labels_2)

# convert to windowed labels
data_labels = np.repeat(sub_labels_2, 14, axis=0)
print(data_labels.shape)
print(data_labels)

#convert to tensor
#ata = torch.tensor(data)
data = torch.from_numpy(data).float()
data_labels = torch.from_numpy(data_labels).float()
print(data.dtype)
print(data.shape)
dataset = TensorDataset(Tensor(data) , Tensor(data_labels))

print(data[29][1])

   Participant_id  Trial  Experiment_id  ...  Dominance  Liking  Familiarity
0               1      1              5  ...       7.19    6.05          4.0
1               1      2             18  ...       6.94    8.01          4.0
2               1      3              4  ...       6.12    8.06          4.0
3               1      4             24  ...       8.01    8.22          4.0
4               1      5             20  ...       7.19    8.13          1.0

[5 rows x 9 columns]
(1280, 2)
[[6.96 3.92]
 [7.23 7.15]
 [4.94 6.01]
 ...
 [8.05 7.09]
 [4.01 7.17]
 [4.08 5.95]]
[2. 3. 1. ... 3. 1. 1.]
(17920,)
[2. 2. 2. ... 1. 1. 1.]
torch.float32
torch.Size([17920, 4, 32, 32])
tensor([[ 1.0000,  0.6335,  0.4222,  ..., -0.0332,  0.3070,  0.1884],
        [ 0.6335,  1.0000,  0.8858,  ..., -0.0963,  0.4528,  0.2397],
        [ 0.4222,  0.8858,  1.0000,  ...,  0.0902,  0.4818,  0.2817],
        ...,
        [-0.0332, -0.0963,  0.0902,  ...,  1.0000,  0.4132,  0.5115],
        [ 0.3070,  0.4528, 

In [11]:
pre_train_size = int(0.9 * len(dataset))
test_size = len(dataset) - pre_train_size
pre_train_set, test_set = torch.utils.data.random_split(dataset, [pre_train_size, test_size])
train_size = int(0.8 * len(pre_train_set))
val_size = len(pre_train_set) - train_size
print(len(pre_train_set))
print(train_size)
print(val_size)
train_set, val_set = torch.utils.data.random_split(pre_train_set, [train_size, val_size])

batch_size = 128
print(len(train_set), len(val_set), len(test_set))
trainloader = torch.utils.data.DataLoader(train_set, batch_size=batch_size,
                                          shuffle=False, num_workers=2)
valloader = torch.utils.data.DataLoader(val_set, batch_size=batch_size,
                                          shuffle=False, num_workers=2)
testloader = torch.utils.data.DataLoader(test_set, batch_size=batch_size,
                                          shuffle=False, num_workers=2)
dataloaders = {
    'train': trainloader,
    'val': valloader,
}

16128
12902
3226
12902 3226 1792


In [12]:
# with auto encoder
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

# skipped autoencoder
model = nn.Sequential(
    nn.Conv2d(4, 32, [3, 1]),
    nn.ReLU(),
    nn.Dropout(),
    nn.Conv2d(32, 64, [3, 1]),
    nn.ReLU(), # Maybe not sure
    nn.Dropout(),
    nn.MaxPool2d([3, 3]),
    nn.Flatten(),
    nn.Linear(5760, 512),
    nn.ReLU(),
    nn.Linear(512, 256),
    nn.ReLU(),
    nn.Linear(256, 4)
)
model.to(device)
loss_func = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
print(model)

cuda:0
Sequential(
  (0): Conv2d(4, 32, kernel_size=[3, 1], stride=(1, 1))
  (1): ReLU()
  (2): Dropout(p=0.5, inplace=False)
  (3): Conv2d(32, 64, kernel_size=[3, 1], stride=(1, 1))
  (4): ReLU()
  (5): Dropout(p=0.5, inplace=False)
  (6): MaxPool2d(kernel_size=[3, 3], stride=[3, 3], padding=0, dilation=1, ceil_mode=False)
  (7): Flatten(start_dim=1, end_dim=-1)
  (8): Linear(in_features=5760, out_features=512, bias=True)
  (9): ReLU()
  (10): Linear(in_features=512, out_features=256, bias=True)
  (11): ReLU()
  (12): Linear(in_features=256, out_features=4, bias=True)
)


In [13]:
since = time.time()
best_model_wts = copy.deepcopy(model.state_dict())
best_loss = 10000.0
best_acc = 0
all_train_loss = []
all_val_loss = []
num_epochs = 100
for epoch in range(num_epochs):
    print('Epoch {}/{}'.format(epoch, num_epochs - 1))
    print('-' * 10)

    # Each epoch has a training and validation phase
    for phase in ['train', 'val']:
        if phase == 'train':
            model.train()  # Set model to training mode
        else:
            model.eval()   # Set model to evaluate mode

        running_loss = 0.0
        running_corrects = 0

        # Iterate over data.
        for inputs, labels in dataloaders[phase]:
            inputs = inputs.to(device)
            labels = labels.to(device)
            #print(inputs)
            #print(labels)
            

            # zero the parameter gradients
            optimizer.zero_grad()

            # forward
            # track history if only in train
            with torch.set_grad_enabled(phase == 'train'):
                outputs = model(inputs)
                #print(outputs)
                _, preds = torch.max(outputs, 1)
                #print(preds)
                #print(outputs.dtype)
                #print(labels.dtype)
                loss = loss_func(outputs, labels.long())
                #print(loss.item())

                # backward + optimize only if in training phase
                if phase == 'train':
                    loss.backward()
                    optimizer.step()

                # statistics
                running_loss += loss.item()

                #print(running_loss)
                
                running_corrects += torch.sum(preds == labels.data) / inputs.size(0)

        epoch_loss = running_loss / len(dataloaders[phase])
        epoch_acc = running_corrects / len(dataloaders[phase])

        print('{} Loss: {:.4f} Acc: {:.4f}'.format(
            phase, epoch_loss, epoch_acc))
        if phase == 'train':
          all_train_loss.append(epoch_loss)
        else:
          all_val_loss.append(epoch_loss)
        # deep copy the model
        # if phase == 'val' and epoch_acc > best_acc:
        #     best_acc = epoch_acc
        #     best_model_wts = copy.deepcopy(model.state_dict())
        if phase == 'val' and epoch_loss < best_loss:
            best_loss = epoch_loss
            best_model_wts = copy.deepcopy(model.state_dict())

    print()

time_elapsed = time.time() - since
print('Training complete in {:.0f}m {:.0f}s'.format(
    time_elapsed // 60, time_elapsed % 60))
print('Best val Loss: {:4f}'.format(best_loss))

# load best model weights
model.load_state_dict(best_model_wts)

# torch.save(model.state_dict(), '/content/drive/MyDrive/CSE 481 Capstone/model_multi.pth')

Epoch 0/99
----------
train Loss: 1.3838 Acc: 0.3469
val Loss: 1.3773 Acc: 0.3580

Epoch 1/99
----------
train Loss: 1.3512 Acc: 0.3603
val Loss: 1.3675 Acc: 0.3730

Epoch 2/99
----------
train Loss: 1.3230 Acc: 0.3787
val Loss: 1.3384 Acc: 0.3799

Epoch 3/99
----------
train Loss: 1.2857 Acc: 0.3941
val Loss: 1.3121 Acc: 0.3931

Epoch 4/99
----------
train Loss: 1.2676 Acc: 0.4031
val Loss: 1.3048 Acc: 0.3862

Epoch 5/99
----------
train Loss: 1.2519 Acc: 0.4168
val Loss: 1.2940 Acc: 0.3904

Epoch 6/99
----------
train Loss: 1.2385 Acc: 0.4225
val Loss: 1.2797 Acc: 0.4011

Epoch 7/99
----------
train Loss: 1.2289 Acc: 0.4259
val Loss: 1.2729 Acc: 0.4018

Epoch 8/99
----------
train Loss: 1.2188 Acc: 0.4326
val Loss: 1.2663 Acc: 0.4060

Epoch 9/99
----------
train Loss: 1.2092 Acc: 0.4417
val Loss: 1.2602 Acc: 0.4069

Epoch 10/99
----------
train Loss: 1.1975 Acc: 0.4450
val Loss: 1.2466 Acc: 0.4162

Epoch 11/99
----------
train Loss: 1.1894 Acc: 0.4511
val Loss: 1.2448 Acc: 0.4174

Ep

<All keys matched successfully>

In [14]:
#autoencoder
print(model)
new_model = nn.Sequential(*list(model.children())[:-5])
new_model.to(device)

Sequential(
  (0): Conv2d(4, 32, kernel_size=[3, 1], stride=(1, 1))
  (1): ReLU()
  (2): Dropout(p=0.5, inplace=False)
  (3): Conv2d(32, 64, kernel_size=[3, 1], stride=(1, 1))
  (4): ReLU()
  (5): Dropout(p=0.5, inplace=False)
  (6): MaxPool2d(kernel_size=[3, 3], stride=[3, 3], padding=0, dilation=1, ceil_mode=False)
  (7): Flatten(start_dim=1, end_dim=-1)
  (8): Linear(in_features=5760, out_features=512, bias=True)
  (9): ReLU()
  (10): Linear(in_features=512, out_features=256, bias=True)
  (11): ReLU()
  (12): Linear(in_features=256, out_features=4, bias=True)
)


Sequential(
  (0): Conv2d(4, 32, kernel_size=[3, 1], stride=(1, 1))
  (1): ReLU()
  (2): Dropout(p=0.5, inplace=False)
  (3): Conv2d(32, 64, kernel_size=[3, 1], stride=(1, 1))
  (4): ReLU()
  (5): Dropout(p=0.5, inplace=False)
  (6): MaxPool2d(kernel_size=[3, 3], stride=[3, 3], padding=0, dilation=1, ceil_mode=False)
  (7): Flatten(start_dim=1, end_dim=-1)
)

In [15]:
#autoencoder
auto_model = nn.Sequential(
    nn.Linear(5760, 128),
    nn.ReLU(),
    nn.Linear(128, 5760)
)
auto_model.to(device)
optimizer2 = torch.optim.Adam(auto_model.parameters(), lr=0.001)

In [16]:
since = time.time()
#best_model_wts = copy.deepcopy(auto_model.state_dict())
best_acc = 0.0
all_train_loss = []
all_val_loss = []
num_epochs = 50
auto_loss_function = nn.MSELoss()
for epoch in range(num_epochs):
    print('Epoch {}/{}'.format(epoch, num_epochs - 1))
    print('-' * 10)

    # Each epoch has a training and validation phase
    for phase in ['train', 'val']:
        if phase == 'train':
            auto_model.train()  # Set model to training mode
        else:
            auto_model.eval()   # Set model to evaluate mode

        running_loss = 0.0
        running_corrects = 0

        # Iterate over data.
        for inputs, labels in dataloaders[phase]:
            inputs = inputs.to(device)
            labels = labels.to(device)
            #print(inputs)
            #print(labels)
            

            # zero the parameter gradients
            optimizer2.zero_grad()
            

            # forward
            # track history if only in train
            with torch.set_grad_enabled(phase == 'train'):
                pre_output = new_model(inputs)
                outputs = auto_model(pre_output)
                # print(inputs.shape)
                # print(pre_output.shape)
                # print(outputs.shape)
                
                loss = auto_loss_function(outputs, pre_output)
                #print(loss.item())

                # backward + optimize only if in training phase
                if phase == 'train':
                    loss.backward()
                    optimizer2.step()

                # statistics
                running_loss += loss.item()

                #print(running_loss)
                
                #running_corrects += torch.sum(preds == labels.data) / inputs.size(0)

        epoch_loss = running_loss / len(dataloaders[phase])
        #epoch_acc = running_corrects / len(dataloaders[phase])

        print('{} Loss: {:.4f}'.format(
            phase, epoch_loss))
        if phase == 'train':
          all_train_loss.append(epoch_loss)
        else:
          all_val_loss.append(epoch_loss)
        # deep copy the model
        # if phase == 'val' and epoch_acc > best_acc:
        #     best_acc = epoch_acc
        #     best_model_wts = copy.deepcopy(model.state_dict())

    print()

time_elapsed = time.time() - since
print('Training complete in {:.0f}m {:.0f}s'.format(
    time_elapsed // 60, time_elapsed % 60))
#print('Best val Acc: {:4f}'.format(best_acc))

# load best model weights
#model.load_state_dict(best_model_wts)

Epoch 0/49
----------
train Loss: 0.0100
val Loss: 0.0078

Epoch 1/49
----------
train Loss: 0.0071
val Loss: 0.0067

Epoch 2/49
----------
train Loss: 0.0066
val Loss: 0.0065

Epoch 3/49
----------
train Loss: 0.0064
val Loss: 0.0063

Epoch 4/49
----------
train Loss: 0.0063
val Loss: 0.0062

Epoch 5/49
----------
train Loss: 0.0062
val Loss: 0.0061

Epoch 6/49
----------
train Loss: 0.0061
val Loss: 0.0061

Epoch 7/49
----------
train Loss: 0.0061
val Loss: 0.0061

Epoch 8/49
----------
train Loss: 0.0061
val Loss: 0.0061

Epoch 9/49
----------
train Loss: 0.0061
val Loss: 0.0061

Epoch 10/49
----------
train Loss: 0.0061
val Loss: 0.0061

Epoch 11/49
----------
train Loss: 0.0061
val Loss: 0.0061

Epoch 12/49
----------
train Loss: 0.0060
val Loss: 0.0061

Epoch 13/49
----------
train Loss: 0.0060
val Loss: 0.0061

Epoch 14/49
----------
train Loss: 0.0061
val Loss: 0.0061

Epoch 15/49
----------
train Loss: 0.0060
val Loss: 0.0061

Epoch 16/49
----------
train Loss: 0.0060
val Loss

In [17]:
#autoencoder
# cnn_model = nn.Sequential(
#     nn.Linear(5760, 512),
#     nn.ReLU(),
#     nn.Linear(512, 256),
#     nn.ReLU(),
#     nn.Linear(256, 128),
#     nn.ReLU(),
#     nn.Linear(128, 4)
# )
dnn_model = nn.Sequential(*list(model.children())[-5:])
optimizer3 = torch.optim.Adam(dnn_model.parameters(), lr=0.001)
dnn_model.to(device)


Sequential(
  (0): Linear(in_features=5760, out_features=512, bias=True)
  (1): ReLU()
  (2): Linear(in_features=512, out_features=256, bias=True)
  (3): ReLU()
  (4): Linear(in_features=256, out_features=4, bias=True)
)

In [18]:
since = time.time()
best_model_wts = copy.deepcopy(dnn_model.state_dict())
best_loss = 1000
all_train_loss = []
all_val_loss = []
num_epochs = 50
for epoch in range(num_epochs):
    print('Epoch {}/{}'.format(epoch, num_epochs - 1))
    print('-' * 10)

    # Each epoch has a training and validation phase
    for phase in ['train', 'val']:
        if phase == 'train':
            dnn_model.train()  # Set model to training mode
        else:
            dnn_model.eval()   # Set model to evaluate mode

        running_loss = 0.0
        running_corrects = 0

        # Iterate over data.
        for inputs, labels in dataloaders[phase]:
            inputs = inputs.to(device)
            labels = labels.to(device)
            #print(inputs)
            #print(labels)
            

            # zero the parameter gradients
            optimizer3.zero_grad()
            
            # forward
            # track history if only in train
            with torch.set_grad_enabled(phase == 'train'):
                pre_outputs = new_model(inputs)
                auto_outputs = auto_model(pre_outputs)
                outputs = dnn_model(pre_outputs)
                #print(outputs)
                _, preds = torch.max(outputs, 1)
                #print(preds)
                #print(outputs.dtype)
                #print(labels.dtype)
                loss = loss_func(outputs, labels.long())
                #print(loss.item())

                # backward + optimize only if in training phase
                if phase == 'train':
                    loss.backward()
                    optimizer3.step()

                # statistics
                running_loss += loss.item()

                #print(running_loss)
                
                running_corrects += torch.sum(preds == labels.data) / inputs.size(0)

        epoch_loss = running_loss / len(dataloaders[phase])
        epoch_acc = running_corrects / len(dataloaders[phase])

        print('{} Loss: {:.4f} Acc: {:.4f}'.format(
            phase, epoch_loss, epoch_acc))
        if phase == 'train':
          all_train_loss.append(epoch_loss)
        else:
          all_val_loss.append(epoch_loss)
        # deep copy the model
        # if phase == 'val' and epoch_acc > best_acc:
        #     best_acc = epoch_acc
        #     best_model_wts = copy.deepcopy(dnn_model.state_dict())
        if phase == 'val' and epoch_loss < best_loss:
            best_loss = epoch_loss
            best_model_wts = copy.deepcopy(model.state_dict())

    print()

time_elapsed = time.time() - since
print('Training complete in {:.0f}m {:.0f}s'.format(
    time_elapsed // 60, time_elapsed % 60))
print('Best val loss: {:4f}'.format(best_loss))

# load best model weights
dnn_model.load_state_dict(best_model_wts)

Epoch 0/49
----------
train Loss: 0.8064 Acc: 0.6677
val Loss: 1.2826 Acc: 0.4688

Epoch 1/49
----------
train Loss: 0.7010 Acc: 0.7202
val Loss: 1.3372 Acc: 0.4739

Epoch 2/49
----------
train Loss: 0.6172 Acc: 0.7577
val Loss: 1.4081 Acc: 0.4685

Epoch 3/49
----------
train Loss: 0.5395 Acc: 0.7943
val Loss: 1.4923 Acc: 0.4763

Epoch 4/49
----------
train Loss: 0.4641 Acc: 0.8317
val Loss: 1.5891 Acc: 0.4745

Epoch 5/49
----------
train Loss: 0.3968 Acc: 0.8642
val Loss: 1.7249 Acc: 0.4754

Epoch 6/49
----------
train Loss: 0.3427 Acc: 0.8869
val Loss: 1.8659 Acc: 0.4743

Epoch 7/49
----------
train Loss: 0.3044 Acc: 0.8988
val Loss: 2.0129 Acc: 0.4736

Epoch 8/49
----------
train Loss: 0.2917 Acc: 0.8956
val Loss: 2.1210 Acc: 0.4607

Epoch 9/49
----------
train Loss: 0.2965 Acc: 0.8880
val Loss: 2.2944 Acc: 0.4646

Epoch 10/49
----------
train Loss: 0.3192 Acc: 0.8772
val Loss: 2.4652 Acc: 0.4733

Epoch 11/49
----------
train Loss: 0.3178 Acc: 0.8779
val Loss: 2.3468 Acc: 0.4709

Ep

RuntimeError: ignored

In [ ]:
datatestiter = iter(testloader)
input_test, labels_test = datatestiter.next()
input_test = input_test.to(device)
labels_test = labels_test.to(device)
new_model.eval()
auto_model.eval()
dnn_model.eval()
one = new_model(input_test)
two = auto_model(one)
output_test = dnn_model(two)
_, preds_test = torch.max(output_test, 1)
print(preds_test.shape)
print(labels_test.shape)
print('predictions', preds_test)
print('labels', labels_test)
print(torch.sum(preds_test == labels_test) / len(preds_test))